In [9]:
#imports  dev F3
import pandas as pd
from helper import pickle_store, pickle_restore
import dateutil
from adem import *
from collections import Counter 


In [10]:
#get email data of first 3 months: data is without date column converted to timedates type. File was stored in prev features

isdev=True # f6 True, f9 False 

#get email data of first 3 months: data is without date column converted to timedates type
if isdev:
    dev_raw_df = pickle_restore("pickle/dev_rawdf_file")
else:
    dev_raw_df = pickle_restore("pickle/log_rawdf_file")


# dev_raw_df = pickle_restore("dev_rawdf_file")

dev_act_feature_df=dev_raw_df[['user', 'date', 'pc', 'activity']] #ONLY CHANGE iS THIS wrt device_fem

#select data only till 1 April (4/1/2010). 
# convert date column in date form to comapre and select small data till desired date.
dev_act_feature_df['date'] = pd.to_datetime(dev_act_feature_df['date'])
# mask = dev_act_feature_df['date'] <= end_d
mask = (dev_act_feature_df['date'] <= end_d) & (dev_act_feature_df['date'] >= start_d)
dev_act_feature_df = dev_act_feature_df.loc[mask]

# select only after office hours rows. 
dev_act_feature_df["date"] = pd.to_datetime(dev_act_feature_df["date"])
time_mask = (dev_act_feature_df['date'].dt.hour >= 19) | (dev_act_feature_df['date'].dt.hour < 7)
dev_act_feature_df = dev_act_feature_df[time_mask]



#create a copy of orig df so that I can separately process for ubp pbp and cbp
dev_act_feature_cmnty_df = dev_act_feature_df.copy()
dev_act_feature_peer_df = dev_act_feature_df.copy()

# 1. in email_freq_feature_df add community and peer column with id as values.
usr_cmnty_map = pickle_restore("pickle/community_louvian_file")
dev_act_feature_cmnty_df['cmnty']=dev_act_feature_cmnty_df.apply(lambda row: usr_cmnty_map[row.user], axis=1)

usr_peer_map = pickle_restore("pickle/eid_role_map_file")
dev_act_feature_peer_df['peer']=dev_act_feature_peer_df.apply(lambda row: usr_peer_map[row.user], axis=1)


In [11]:
dev_act_feature_df

,user,date,pc,activity
131068,EIS0041,2010-06-01 00:47:01,PC-3124,Connect
131069,BSS0369,2010-06-01 00:47:38,PC-8508,Connect
131070,BSS0369,2010-06-01 00:47:58,PC-8508,Disconnect
131071,EIS0041,2010-06-01 00:51:34,PC-3124,Disconnect
131072,MOS0047,2010-06-01 01:23:00,PC-6688,Connect
...,...,...,...,...
305590,FMG0527,2010-12-30 22:41:33,PC-4256,Disconnect
305591,FMG0527,2010-12-30 23:08:43,PC-4256,Connect
305592,FMG0527,2010-12-30 23:12:54,PC-4256,Disconnect
305593,FMG0527,2010-12-30 23:34:33,PC-4256,Connect


In [12]:
# Convert date back to string format
dev_act_feature_df['date']=dev_act_feature_df['date'].astype(str)
dev_act_feature_cmnty_df['date']=dev_act_feature_cmnty_df['date'].astype(str)
dev_act_feature_peer_df['date']=dev_act_feature_peer_df['date'].astype(str)


In [13]:
# Now data is ready to generate prep_dic using groupby, UBP. Feature = num_activity
def prep_ubp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False) #NOTE converts day to day first year-dd-mm
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    #creates user, date wise groups and each internal group is a dataframe.
    # grp=tmp_email_freq_feature_df.groupby(['user', 'date'])

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['user', 'date']).activity.agg('count').to_frame('num_usage_posthr').reset_index()

    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['user'])

    email_freq_feature_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        email_freq_feature_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()

    return email_freq_feature_dic


def prep_cbp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['cmnty', 'date']).activity.agg('count').to_frame('num_usage_posthr').reset_index()

    cmnty_len = Counter(usr_cmnty_map.values())
    #make values of email freq avg of community
    df['num_usage_posthr'] = df.apply( lambda x: x.num_usage_posthr/cmnty_len[x.cmnty], axis=1)



    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['cmnty'])

    feature_cmnty_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        feature_cmnty_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()



    return feature_cmnty_dic

def prep_pbp_dic(df):
    #convert date column of srt type to Timestamp type.
    df['date'] = df['date'].apply(dateutil.parser.parse, dayfirst=False)
    #now convert date column of Timestamptype to only date values and remove hr:mm:ss
    df['date'] = pd.to_datetime(df['date']).dt.date

    # create user, date wise groups and count unique dates then add a separate column for the counts
    df=df.groupby(['peer', 'date']).activity.agg('count').to_frame('num_usage_posthr').reset_index()

    cmnty_len = Counter(usr_peer_map.values())
    #make values of email freq avg of community
    df['num_usage_posthr'] = df.apply( lambda x: x.num_usage_posthr/cmnty_len[x.peer], axis=1)

    #groupby user now To populate a dictionary for each user.
    grp=df.groupby(['peer'])

    feature_peer_dic={} #user:df with feature value
    #iterating groups
    for name, group in grp:
        # print (name)
        # print (group)
        feature_peer_dic[name]=group
        # ldf = group.groupby(['date'], as_index=False)['cntr'].size()
    return feature_peer_dic



In [14]:
ubp_dev_act_feature_df = dev_act_feature_df.copy()
ubp_dev_act_feature_dic = prep_ubp_dic(ubp_dev_act_feature_df)


cbp_dev_act_feature_df = dev_act_feature_cmnty_df.copy()
cbp_dev_act_feature_dic = prep_cbp_dic(cbp_dev_act_feature_df)


pbp_dev_act_feature_df = dev_act_feature_peer_df.copy()
pbp_dev_act_feature_dic = prep_pbp_dic(pbp_dev_act_feature_df)



#store all preped dic
if isdev:
    pickle_store("pickle/f6/f6_ubp_file", ubp_dev_act_feature_dic)
    pickle_store("pickle/f6/f6_pbp_file", pbp_dev_act_feature_dic)
    pickle_store("pickle/f6/f6_cbp_file", cbp_dev_act_feature_dic)
else:
    pickle_store("pickle/f9/f9_ubp_file", ubp_dev_act_feature_dic)
    pickle_store("pickle/f9/f9_pbp_file", pbp_dev_act_feature_dic)
    pickle_store("pickle/f9/f9_cbp_file", cbp_dev_act_feature_dic)



In [18]:
from adem import *
count = 0
feature_name = 'num_usage_posthr'

print('PBP')
for key in ubp_dev_act_feature_dic:
    usr=key #'AAM0658'
    debug_df = ubp_dev_act_feature_dic[usr]
    debug_peer_df = pbp_dev_act_feature_dic[usr_peer_map[usr]]

    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_peer = debug_peer_df.copy()
    
    uname = key
    print(key)
    anom_calc_pbp(df, df_peer, feature_name, uname, ws=10, sig=2)
    
    # count += 1
    # if count%5 == 0:
    #     break

print('CBP')
for key in ubp_dev_act_feature_dic:
    usr=key
    debug_df = ubp_dev_act_feature_dic[usr]
    debug_cmnty_df = cbp_dev_act_feature_dic[usr_cmnty_map[usr]]

    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    df_cmnty = debug_cmnty_df.copy()

    uname = key
    print(key)
    anom_calc_cbp(df, df_cmnty, feature_name, uname, ws=10, sig=2)
    
    # count += 1
    # if count%5 == 0:
    #     break


print('UBP')
# email_freq_feature_dic.keys()
# Feature data format:  user: df [user, date, featurecolmn]
for key in ubp_dev_act_feature_dic:
    usr=key
    debug_df = ubp_dev_act_feature_dic[usr]


    #do a depp copy of preped feature dataframe
    df = debug_df.copy()
    print(key)
    anom_calc_ubp(df, feature_name, usr, ws=5, sig=2)
    
    # count += 1
    # if count%10 == 0:
    #     break

PBP
AAM0658
0.10786673988635857
64
AAN0823
0.08357845578958407
64
ABM0845
0.08357845578958407
64
ACC0042
2.3021494139791425
10
ACV0812
0.23254489754122548
67
ADC0391
0.17882828472460952
42
ADD0846
0.08357845578958407
64
ADF0397
0.17882828472460952
34
AES0827
0.08357845578958407
64
AGB0643
0.10786673988635857
64
AHD0848
0.08357845578958407
70
AHG0634
0.10786673988635857
64
AIP0394
0.17882828472460952
41
AIS0405
0.17882828472460952
41
AJC0840
0.08357845578958407
64
AJD0074
0.07040598125168901
43
AJF0370
2.3021494139791425
14
AJH0175
0.026917675815608413
14
AJL0462
0.08357845578958407
10
AKC0924
0.14756986287359722
64
AKR0057
0.07040598125168901
48
ALC0788
0.08478844709339436
64
ALH0402
0.17882828472460952
38
AMM0014
0.11677967913520755
5
AMR0318
0.026917675815608413
9
AMR0400
0.17882828472460952
38
AMW0392
0.16575938438031054
65
ANM0123
0.07040598125168901
32
AOK0844
0.08357845578958407
90
AQD0820
0.17882828472460952
64
ARB0834
0.08357845578958407
64
ASG0692
0.16575938438031054
42
ATE086